In [1]:
import requests
import json
import pandas as pd
import numpy as np
import pymysql
import os

In [2]:
# 1. 영화 id 정보추출을 위해 영화정보 파일 가져오기 

tmdb_movie_list = pd.read_csv('../data-files/total_tmdbmovielist.csv',index_col=0,lineterminator='\n') 

/Users/parkseonghun/opt/miniconda3/envs/kdigital-busan/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,4,7,12,17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
tmdb_movie_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 652715 entries, 0 to 652714
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   adult                  652687 non-null  object 
 1   backdrop_path          154275 non-null  object 
 2   belongs_to_collection  12081 non-null   object 
 3   budget                 652686 non-null  object 
 4   genres                 652684 non-null  object 
 5   homepage               71779 non-null   object 
 6   id                     652685 non-null  object 
 7   imdb_id                403838 non-null  object 
 8   original_language      652686 non-null  object 
 9   original_title         652659 non-null  object 
 10  overview               25451 non-null   object 
 11  popularity             652648 non-null  object 
 12  poster_path            440667 non-null  object 
 13  production_companies   652648 non-null  object 
 14  production_countries   652617 non-nu

In [4]:
# 1. 영화테이블에서 장르id만 추출하여 genre_ids칼럼을 만들기  

error_list_x = []

def genres_str_to_list(genres_str):
    import json
    # print(genres_str)
    genre_id_list = []
    try :
        genres_list = json.loads(genres_str.replace("\'", "\""))
        for genre in genres_list:
            genre_id_list.append(genre['id'])

    except:
        error_list_x.append(genres_str)
    
    return genre_id_list

tmdb_movie_list['genre_ids'] = tmdb_movie_list["genres"].map(genres_str_to_list)

tmdb_movie_list.tail(5)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,genre_ids
652710,False,NaN,NaN,0.0,"[{'id': 35, 'name': '코미디'}]",NaN,699995.0,tt0300432,cs,Roztržené foto,...,0.0,0.0,"[{'english_name': 'No Language', 'iso_639_1': ...",Released,NaN,Roztržené foto,False,0.0,0.0,[35]
652711,False,NaN,NaN,0.0,[],https://www.yesterway.org/,699996.0,NaN,en,Yesterway,...,0.0,107.0,[],Released,NaN,Yesterway,False,0.0,0.0,[]
652712,False,NaN,NaN,0.0,[],NaN,699997.0,tt4420000,en,Kalopsia,...,0.0,12.0,[],Released,NaN,Kalopsia,False,0.0,0.0,[]
652713,False,NaN,NaN,0.0,[],NaN,699998.0,tt11872724,es,Poderoso Victoria,...,0.0,0.0,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,NaN,Poderoso Victoria,False,5.0,1.0,[]
652714,False,NaN,NaN,0.0,[],NaN,699999.0,tt0324364,fr,Is Dead - Portrait incomplet de Gertrude Stein,...,0.0,47.0,[],Released,NaN,Is Dead - Portrait incomplet de Gertrude Stein,False,0.0,0.0,[]


In [5]:
# genre_ids가 없는 행 제거 

print( type(tmdb_movie_list['genre_ids'][0]))
tmdb_movie_list['genre_ids'].map(lambda x : True if len(x) > 0 else False)
tmdb_movie_list_1=tmdb_movie_list[tmdb_movie_list['genre_ids'].map(lambda x : True if len(x) > 0 else False)]
tmdb_movie_list_1


<class 'list'>


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,genre_ids
0,False,NaN,NaN,0.0,"[{'id': 27, 'name': '공포'}, {'id': 9648, 'name'...",NaN,280000.0,tt2396436,en,Lemon Tree Passage,...,0.0,84.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Lemon Tree Passage,False,4.1,75.0,"[27, 9648, 53]"
1,False,NaN,NaN,0.0,"[{'id': 99, 'name': '다큐멘터리'}]",NaN,280001.0,tt0163823,en,The Second Gun,...,0.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,The Second Gun,False,0.0,0.0,[99]
2,False,/bLI7uIY9Y1vDkx1Bt6Crn2NJELL.jpg,NaN,0.0,"[{'id': 53, 'name': '스릴러'}, {'id': 27, 'name':...",NaN,280002.0,tt2714380,en,The Remaining,...,1169603.0,88.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,리메이닝,False,5.5,246.0,"[53, 27]"
3,False,NaN,NaN,0.0,"[{'id': 16, 'name': '애니메이션'}]",NaN,280003.0,tt0231466,cs,Diskjockey,...,0.0,10.0,[],Released,NaN,Diskjockey,False,5.8,13.0,[16]
4,False,NaN,NaN,0.0,"[{'id': 18, 'name': '드라마'}, {'id': 10402, 'nam...",NaN,280004.0,tt0068662,en,The Great Waltz,...,0.0,135.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,The Great Waltz,False,0.0,0.0,"[18, 10402, 10749]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652703,False,NaN,NaN,0.0,"[{'id': 16, 'name': '애니메이션'}]",NaN,699986.0,NaN,en,Goops,...,0.0,1.0,[],Released,NaN,Goops,False,6.0,1.0,[16]
652704,False,NaN,NaN,0.0,"[{'id': 99, 'name': '다큐멘터리'}]",NaN,699988.0,tt5256726,en,Favre Returns,...,0.0,43.0,[],Released,NaN,Favre Returns,False,8.0,1.0,[99]
652707,False,NaN,NaN,0.0,"[{'id': 18, 'name': '드라마'}]",NaN,699992.0,NaN,cs,Moderní Magdalena,...,0.0,0.0,"[{'english_name': 'No Language', 'iso_639_1': ...",Released,NaN,Moderní Magdalena,False,0.0,0.0,[18]
652708,False,NaN,NaN,0.0,"[{'id': 99, 'name': '다큐멘터리'}]",NaN,699993.0,NaN,es,La ciudad de los signos,...,0.0,63.0,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,NaN,La ciudad de los signos,False,0.0,0.0,[99]


In [7]:
movie_genre_ids_df=tmdb_movie_list_1[['id','genre_ids']]
movie_genre_ids_df.head()

,id,genre_ids
0,280000.0,"[27, 9648, 53]"
1,280001.0,[99]
2,280002.0,"[53, 27]"
3,280003.0,[16]
4,280004.0,"[18, 10402, 10749]"


In [8]:
##영화_장르 연결테이블 2)movie_id, genre_id삽입

movie_genre_ids_df=tmdb_movie_list_1[['id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()
cursor.execute('DELETE FROM movie_genre')

error_list2 = []
for movie_genre_ids in movie_genre_ids_df.values:
    #print(movie_genre_ids[1])
    #break
    current_genre_id = None
    try:
        movie_id = movie_genre_ids[0]
        for genre_id in movie_genre_ids[1]:
            current_genre_id = genre_id
            sql= "INSERT INTO movie_genre VALUES (%s, %s)"
            cursor.execute(sql, (movie_id, genre_id))
            # print((movie_id, genre_id))
    except:
        error_list2.append((movie_id, current_genre_id))
conn.commit()
cursor.close()
print('close')
conn.close()

close


In [9]:
print(len(error_list2))
error_list2

81293


[(286976.0, 35),
 (632473.0, 10751),
 (632601.0, 27),
 (632682.0, 18),
 (633181.0, 18),
 (633933.0, 99),
 (634401.0, 18),
 (635183.0, 80),
 (639957.0, 35),
 (504377.0, 18),
 (507603.0, 18),
 (508492.0, 53),
 (509188.0, 99),
 (385781.0, 18),
 (388008.0, 99),
 (401065.0, 27),
 (401067.0, 53),
 (401068.0, 35),
 (401069.0, 35),
 (401070.0, 18),
 (401072.0, 99),
 (401078.0, 28),
 (401081.0, 80),
 (401084.0, 35),
 (401085.0, 35),
 (401089.0, 35),
 (401090.0, 28),
 (401092.0, 35),
 (401093.0, 18),
 (401097.0, 10402),
 (401098.0, 18),
 (401099.0, 80),
 (401100.0, 18),
 (401101.0, 10752),
 (401104.0, 18),
 (401105.0, 80),
 (401106.0, 12),
 (401107.0, 18),
 (401108.0, 18),
 (401111.0, 35),
 (401112.0, 35),
 (401114.0, 99),
 (401115.0, 18),
 (401116.0, 18),
 (401118.0, 99),
 (401120.0, 99),
 (401121.0, 53),
 (401122.0, 18),
 (401123.0, 53),
 (401128.0, 16),
 (401130.0, 18),
 (401131.0, 53),
 (401132.0, 28),
 (401133.0, 53),
 (401134.0, 12),
 (401136.0, 27),
 (401139.0, 28),
 (401140.0, 35),
 (401